In [ ]:
# We will add the below meta data

# > n_tokens: Number of tokens in the text.

# > n_unique_tokens: Number of unique tokens in the text.

# > n_sentences: Number of sentences in the text.

# > perplexity: Measures the text complexity, vocabulary diversity and unpredictability. Lower scores suggest that the model finds the text more predictable, while a higher perplexity score means the model finds the text less predictable.

# > entropy: Indicates text randomness or uncertainty. Higher scores denote varied, unpredictable language use.

# > flesch_reading_ease: A readability test designed to indicate how easy an English text is to understand, based on sentence length and syllable count per word. Higher scores mean that is easier to read, while lower scores indicate complexity.

In [ ]:
import argilla as rg 
from argilla.client.feedback.integrations.textdescriptives import TextDescriptivesExtractor
from datasets import load_dataset

In [ ]:
rg.init(
    api_url="",
    api_key="owner.apikey"
)

In [ ]:
hf_arg_ds = load_dataset("squad", split='train').select(range(100))
hf_arg_ds

In [ ]:
fb_ds = rg.FeedbackDataset.for_question_answering(
    use_markdown=True,
    guidelines="This is a Question Answerine Pipeline",
    metadata_properties=None,
    vectors_settings=None
)
fb_ds

In [ ]:
records = [
    rg.FeedbackRecord(
        fields={"question": record['question'],
                "context": record['context']}
    )
    for record in hf_arg_ds
]

In [ ]:
tde = TextDescriptivesExtractor(
    model="en",
    metrics=None,
    visible_for_annotators=False,
    show_progress=True
)

In [ ]:
update_rec = tde.update_records(records, fields=['question'])

In [ ]:
update_rec[10].metadata

In [ ]:
fb_ds.add_records(update_rec)

In [ ]:
rmote_ds = fb_ds.push_to_argilla(name="squad_withmeta", workspace='hfgilla')

In [ ]:
tde_context = TextDescriptivesExtractor(
    model = "en",
    metrics = ["descriptive_stats", "readability"],
    visible_for_annotators = True,
    show_progress = True,
)

tde.update_dataset(rmote_ds, fields=['context'])

In [ ]:
rmote_ds.records[0].metadata